In [6]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

# Exercise

Go to http://books.toscrape.com/, Using what you have learned create a csv file the contains all the books found in the website. The csv file should contain the following:

- Title
- Price
- Description
- Availability

Code guides have been provided to help you in creating the web scraper. Found below is the `get_title_links_and_next_page` this function returns 2 things book urls in a page and link to the next page. The idea here is to collect first all the book links available in the website and store the links in the `title_links` variable **(5 points)**

In [8]:
base_url = "http://books.toscrape.com/"

def get_title_links_and_next_page(page_url):
    list_links = [] 
    page = urlopen(base_url)
    soup = BeautifulSoup(page, 'html.parser')
    for article in soup.find_all('article'):
        if "catalogue" not in article.find("a")['href']:
            url = base_url + "catalogue/" + article.find("a")['href']
        else:
            url = base_url + article.find("a")['href']
        list_links.append(url)
    
    try:
        next_url = base_url +  li["next"].find("a")["href"]
    except:
        next_url = None

    return (list_links, next_url)

res = get_title_links_and_next_page('http://books.toscrape.com/index.html')
title_links = res[0]

while res[1]:
    if "catalogue" not in res[1]:
        page_url = base_url + "catalogue/" + res[1]
    else:
        page_url = base_url + res[1]
    res = get_title_links_and_next_page(page_url)
    title_links += list_links

title_links

['http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
 'http://books.toscrape.com/catalogue/soumission_998/index.html',
 'http://books.toscrape.com/catalogue/sharp-objects_997/index.html',
 'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html',
 'http://books.toscrape.com/catalogue/the-requiem-red_995/index.html',
 'http://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'http://books.toscrape.com/catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'http://books.toscrape.com/catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'http://books.toscrape.com/catalogue/the-black-maria_991/index.html',
 'http://books.toscrape.com/catalogue/starving-hearts-triangular-trade-trilogy-1_990

Once you have a list of all the available book links we loop through the links and use the 4 functions `get_title`, `get_price`, `get_description`, `get_availability` to retrieve the book information. **(10 points)**

In [47]:
def get_title(soup):
    return soup.find("h1")

def get_price(soup):
    return soup.find("p", attrs={'class':'price_color'})
    
def get_description(soup):
    return soup.find("div", attrs={"id" : "product_description"}).next_sibling.next_sibling

def get_availability(soup):
    available = soup.find('p', attrs={'class':'price_color'}).next_sibling.next_sibling
    return available.find('i').next_sibling

book_data = []
for title_link in title_links: 
    page1 = urlopen(title_link)
    soup = BeautifulSoup(page1, 'html.parser')
    
    title = get_title(soup)
    price = get_price(soup)
    description = get_description(soup)
    availability = get_availability(soup)
    
    book_data += [[title, price, description, availability.strip()]]

In [49]:
df = pd.DataFrame(book_data)
df.columns = ['title', 'price', 'description', 'availability']
display(df.head())
df.to_csv("booked_scrape.csv")

,title,price,description,availability
0,[A Light in the Attic],[£51.77],[It's hard to imagine a world without A Light ...,In stock (22 available)
1,[Tipping the Velvet],[£53.74],"[""Erotic and absorbing...Written with starling...",In stock (20 available)
2,[Soumission],[£50.10],"[Dans une France assez proche de la nôtre, un ...",In stock (20 available)
3,[Sharp Objects],[£47.82],"[WICKED above her hipbone, GIRL across her hea...",In stock (20 available)
4,[Sapiens: A Brief History of Humankind],[£54.23],[From a renowned historian comes a groundbreak...,In stock (20 available)
